In [135]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
import nltk
import re
import gensim
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

In [3]:
emo = pd.read_csv('emobank.csv')

In [4]:
emo.head()

,id,split,V,A,D,text
0,110CYL068_1036_1079,train,3.00,3.00,3.20,"Remember what she said in my last letter? """
1,110CYL068_1079_1110,test,2.80,3.10,2.80,If I wasn't working here.
2,110CYL068_1127_1130,train,3.00,3.00,3.00,".."""
3,110CYL068_1137_1188,train,3.44,3.00,3.22,Goodwill helps people get off of public assist...
4,110CYL068_1189_1328,train,3.55,3.27,3.46,Sherry learned through our Future Works class ...


In [5]:
def assign(x):
  if x == 'dev' or x == 'test':
    return True
  else:
    return False

In [6]:
emo['split'] = emo['split'].apply(assign)

In [7]:
emo['split'].value_counts()

False    8062
True     2000
Name: split, dtype: int64

In [8]:
stop_words =['i','me','my','myself','we','our','ours','ourselves','you','your','yours','yourself',
            'yourselves','he','him','his','himself','she','her','hers','herself','it','its','itself',
            'they','them','their','theirs','themselves','what','which','who','whom','this','that',
            'these','those','am','is','are','was','were','be','been','being','have','has','had',
            'having','do','does','did','doing','a','an','the','and','but','if','or','because','as',
            'until','while','of','at','by','for','with','about','against','between','into','through',
            'during','before','after','above','below','to','from','up','down','in','out','on','off',
            'over','under','again','further','then','once','here','there','when','where','why','how',
            'all','any','both','each','few','more','most','other','some','such','no','nor','not',
            'only','own','same','so','than','too','very','s','t','can','will','just','don','should',
            'now','uses','use','using','used','one','also']


In [9]:
def preprocess(data):
    reviews_tokens = []
    for review in data:
        review = review.lower() #Convert to lower-case words
        raw_word_tokens = re.findall(r'(?:\w+)', review,flags = re.UNICODE) #remove pontuaction
        word_tokens = [w for w in raw_word_tokens if not w in stop_words] # do not add stop words
        reviews_tokens.append(word_tokens)
    return reviews_tokens #return all tokens

In [10]:
emo['text'] = preprocess(list(emo['text']))

In [11]:
emo.head()

,id,split,V,A,D,text
0,110CYL068_1036_1079,False,3.00,3.00,3.20,"[remember, said, last, letter]"
1,110CYL068_1079_1110,True,2.80,3.10,2.80,"[wasn, working]"
2,110CYL068_1127_1130,False,3.00,3.00,3.00,[]
3,110CYL068_1137_1188,False,3.44,3.00,3.22,"[goodwill, helps, people, get, public, assista..."
4,110CYL068_1189_1328,False,3.55,3.27,3.46,"[sherry, learned, future, works, class, could,..."


In [12]:
X = list(emo['text'])
# let X be a list of tokenized texts (i.e. list of lists of tokens)
model = gensim.models.Word2Vec(X, size=100)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

C:\Users\ZheMin\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  after removing the cwd from sys.path.


In [119]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = 100

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [120]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = 100

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [121]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier

etree_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200))])
etree_w2v_tfidf = Pipeline([
    ("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200))])

In [122]:
def transform(X,word2vec):
    dim = 100
    return np.array([
        np.mean([word2vec[w] for w in words if w in word2vec]
                or [np.zeros(dim)], axis=0)
        for words in X
    ])

In [123]:
def sum_transform(X,word2vec):
    dim = 100
    return np.array([
        np.sum([word2vec[w] for w in words if w in word2vec]
                or [np.zeros(dim)], axis=0)
        for words in X
    ])

## Method 1: take mean of word embeddings

In [154]:
emo_test=emo.loc[emo['split']==True]
emo_train=emo.loc[emo['split']==False]
x_train = transform(emo_train['text'],w2v)
x_test = transform(emo_test['text'],w2v)

In [155]:
clf_V = MLPRegressor(solver='sgd', alpha=1e-5, learning_rate = 'adaptive',
                     hidden_layer_sizes=(40, 20), random_state=1, max_iter = 2000)
clf_V.fit(x_train,emo_train.V)

clf_A = MLPRegressor(solver='sgd', alpha=1e-5, learning_rate = 'adaptive',
                     hidden_layer_sizes=(40, 20), random_state=1, max_iter = 2000)
clf_A.fit(x_train,emo_train.A)

clf_D = MLPRegressor(solver='sgd', alpha=1e-5, learning_rate = 'adaptive',
                     hidden_layer_sizes=(40, 20), random_state=1, max_iter = 2000)
clf_D.fit(x_train,emo_train.D)

MLPRegressor(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(40, 20), learning_rate='adaptive',
             learning_rate_init=0.001, max_iter=2000, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=1, shuffle=True, solver='sgd', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [156]:
V_pred = clf_V.predict(x_test)
print("RMSE for V:",np.sqrt(mean_squared_error(emo_test.V, V_pred)))
A_pred = clf_A.predict(x_test)
print("RMSE for A:",np.sqrt(mean_squared_error(emo_test.A, A_pred)))
D_pred = clf_D.predict(x_test)
print("RMSE for D:",np.sqrt(mean_squared_error(emo_test.D, D_pred)))

RMSE for V: 0.34334346005690697
RMSE for A: 0.24913387803853304
RMSE for D: 0.20710904640617162


## Method 2: take sum of word embeddings

In [157]:
emo_test=emo.loc[emo['split']==True]
emo_train=emo.loc[emo['split']==False]
x_train = sum_transform(emo_train['text'],w2v)
x_test = sum_transform(emo_test['text'],w2v)

In [158]:
clf_V = MLPRegressor(solver='sgd', alpha=1e-5, learning_rate = 'adaptive',
                     hidden_layer_sizes=(40, 20), random_state=1, max_iter = 2000)
clf_V.fit(x_train,emo_train.V)

clf_A = MLPRegressor(solver='sgd', alpha=1e-5, learning_rate = 'adaptive',
                     hidden_layer_sizes=(40, 20), random_state=1, max_iter = 2000)
clf_A.fit(x_train,emo_train.A)

clf_D = MLPRegressor(solver='sgd', alpha=1e-5, learning_rate = 'adaptive',
                     hidden_layer_sizes=(40, 20), random_state=1, max_iter = 2000)
clf_D.fit(x_train,emo_train.D)

MLPRegressor(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(40, 20), learning_rate='adaptive',
             learning_rate_init=0.001, max_iter=2000, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=1, shuffle=True, solver='sgd', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [163]:
V_pred = clf_V.predict(x_test)
print("RMSE for V:",np.sqrt(mean_squared_error(emo_test.V, V_pred)))
A_pred = clf_A.predict(x_test)
print("RMSE for A:",np.sqrt(mean_squared_error(emo_test.A, A_pred)))
D_pred = clf_D.predict(x_test)
print("RMSE for D:",np.sqrt(mean_squared_error(emo_test.D, D_pred)))

RMSE for V: 0.3458218124052251
RMSE for A: 0.2502452449486823
RMSE for D: 0.21046637538693508


## Testing individual phrases

In [167]:
sample_text = "A fundraiser dedicated to you and you having fun"

In [175]:
clf_V.predict(np.mean(transform(preprocess([sample_text])[0],w2v),axis=0).reshape(1, -1))

array([2.97369838])

In [176]:
clf_A.predict(np.mean(transform(preprocess([sample_text])[0],w2v),axis=0).reshape(1, -1))

array([3.06400376])

In [177]:
clf_D.predict(np.mean(transform(preprocess([sample_text])[0],w2v),axis=0).reshape(1, -1))

array([3.07429187])